# Generating Word Embeddings - Lab

## Introduction

In this lab, you'll learn how to generate word embeddings by training a Word2Vec model, and then embedding layers into deep neural networks for NLP!

## Objectives

You will be able to:

- Train a Word2Vec model and transform words into vectors 
- Obtain most similar words by using methods associated with word vectors 


## Getting Started

In this lab, you'll start by creating your own word embeddings by making use of the Word2Vec model. Then, you'll move onto building neural networks that make use of **_Embedding Layers_** to accomplish the same end-goal, but directly in your model. 

As you've seen, the easiest way to make use of Word2Vec is to import it from the [Gensim Library](https://radimrehurek.com/gensim/). This model contains a full implementation of Word2Vec, which you can use to begin training immediately. For this lab, you'll be working with the [News Category Dataset from Kaggle](https://www.kaggle.com/rmisra/news-category-dataset/version/2#_=_).  This dataset contains headlines and article descriptions from the news, as well as categories for which type of article they belong to.

Run the cell below to import everything you'll need for this lab. 

In [1]:
import pandas as pd
import numpy as np
np.random.seed(0)
from gensim.models import Word2Vec
from nltk import word_tokenize


Now, import the data. The data is stored in the file `'News_Category_Dataset_v2.json'`.  This file is compressed, so that it can be more easily stored in a GitHub repo. **_Make sure to unzip the file before continuing!_**

In the cell below, use the `read_json()` function from Pandas to read the dataset into a DataFrame. Be sure to include the parameter `lines=True` when reading in the dataset!

Once you've imported the data, inspect the first few rows of the DataFrame to see what your data looks like. 

In [3]:
df = pd.read_json('News_Category_Dataset_v2.json', lines=True)

## Preparing the Data

Since you're working with text data, you need to do some basic preprocessing including tokenization. Notice from the data sample that two different columns contain text data -- `headline` and `short_description`. The more text data your Word2Vec model has, the better it will perform. Therefore, you'll want to combine the two columns before tokenizing each comment and training your Word2Vec model. 

In the cell below:

* Create a column called `'combined_text'` that consists of the data from the `'headline'` column plus a space character (`' '`) plus the data from the `'short_description'` column 
* Use the `'combined_text'` column's `.map()` method and pass in `word_tokenize`. Store the result returned in `data` 

In [4]:
df['combined_text'] = df['headline'] + " " + df['short_description']
data = df['combined_text'].map(word_tokenize)

Inspect the first 5 items in `data` to see how everything looks. 

In [5]:
data[:5]

0    [There, Were, 2, Mass, Shootings, In, Texas, L...
1    [Will, Smith, Joins, Diplo, And, Nicky, Jam, F...
2    [Hugh, Grant, Marries, For, The, First, Time, ...
3    [Jim, Carrey, Blasts, 'Castrato, ', Adam, Schi...
4    [Julianna, Margulies, Uses, Donald, Trump, Poo...
Name: combined_text, dtype: object

Notice that although the words are tokenized, they are still in the same order they were in as headlines. This is important, because the words need to be in their original order for Word2Vec to establish the meaning of them. Remember that for a Word2Vec model you can specify a  **_Window Size_** that tells the model how many words to take into consideration at one time. 

If your window size was 5, then the model would start by looking at the words "Will Smith joins Diplo and", and then slide the window by one, so that it's looking at "Smith joins Diplo and Nicky", and so on, until it had completely processed the text example at index 1 above. By doing this for every piece of text in the entire dataset, the Word2Vec model learns excellent vector representations for each word in an **_Embedding Space_**, where the relationships between vectors capture semantic meaning (recall the vector that captures gender in the previous "king - man + woman = queen" example you saw).

Now that you've prepared the data, train your model and explore a bit!

## Training the Model

Start by instantiating a Word2Vec Model from `gensim`. 

In the cell below:

* Create a `Word2Vec` model and pass in the following arguments:
    * The dataset we'll be training on, `data`
    * The size of the word vectors to create, `size=100`
    * The window size, `window=5`
    * The minimum number of times a word needs to appear in order to be counted in  the model, `min_count=1` 
    * The number of threads to use during training, `workers=4`

In [10]:
model = Word2Vec(data, size=100, window=5, min_count=1, workers=4)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.77 µs
CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.77 µs


Now, that you've instantiated Word2Vec model, train it on your text data. 

In the cell below:

* Call the `.train()` method on your model and pass in the following parameters:
    * The dataset we'll be training on, `data`
    * The `total_examples`  of sentences in the dataset, which you can find in `model.corpus_count` 
    * The number of `epochs` you want to train for, which we'll set to `10`

In [14]:
%timeit
model.train(data, total_examples=model.corpus_count, epochs=10)

(55561040, 67352790)

Great! You now have a fully trained model! The word vectors themselves are stored in the `Word2VecKeyedVectors` instance, which is stored in the `.wv` attribute. To simplify this, restore this object inside of the variable `wv` to save yourself some keystrokes down the line. 

In [15]:
wv = model.wv

## Examining Your Word Vectors

Now that you have a trained Word2Vec model, go ahead and explore the relationships between some of the words in the corpus! 

One cool thing you can use Word2Vec for is to get the most similar words to a given word. You can do this by passing in the word to `wv.most_similar()`. 

In the cell below, try getting the most similar word to `'Texas'`.

In [16]:
wv.most_similar('Texas')

[('Ohio', 0.7644984722137451),
 ('California', 0.7589712142944336),
 ('Florida', 0.7463998794555664),
 ('Illinois', 0.7440139055252075),
 ('Oregon', 0.7392310500144958),
 ('Arkansas', 0.7191222906112671),
 ('Georgia', 0.7187831997871399),
 ('Massachusetts', 0.7172841429710388),
 ('Oklahoma', 0.713620662689209),
 ('Arizona', 0.7134082317352295)]

Interesting! All of the most similar words are also states. 

You can also get the least similar vectors to a given word by passing in the word to the `.most_similar()` method's `negative` parameter. 

In the cell below, get the least similar words to `'Texas'`.

In [17]:
wv.most_similar(negative='Texas')

[('bayou', 0.42122185230255127),
 ('Recognize', 0.3963299095630646),
 ('Memorialize', 0.3952925503253937),
 ('Assert', 0.38879820704460144),
 ('Tasks', 0.3879568874835968),
 ('Teenager', 0.37863337993621826),
 ('Parent/Grandparent', 0.3748236894607544),
 ('Animal-rescue', 0.37081387639045715),
 ('Discussing', 0.3679965138435364),
 ('much-vaunted', 0.36163783073425293)]

This seems like random noise. It is a result of the way Word2Vec is computing the similarity between word vectors in the embedding space. Although the word vectors closest to a given word vector are almost certainly going to have similar meaning or connotation with your given word, the word vectors that the model considers 'least similar' are just the word vectors that are farthest away, or have the lowest cosine similarity. It's important to understand that while the closest vectors in the embedding space will almost certainly share some level of semantic meaning with a given word, there is no guarantee that this relationship will hold at large distances. 

You can also get the vector for a given word by passing in the word as if you were passing in a key to a dictionary. 

In the cell below, get the word vector for `'Texas'`.

In [18]:
wv['Texas']

array([ 1.2307298 ,  0.13335215,  1.8506498 ,  0.05880688, -0.9987183 ,
       -0.35415518,  1.9282831 , -0.8814621 , -1.9630878 , -0.8603534 ,
        3.42102   ,  3.233512  ,  0.5058242 , -2.1061041 ,  0.55476046,
        0.668391  ,  0.8964596 , -0.9279085 ,  0.1142942 ,  1.163279  ,
       -0.64664924, -0.5857592 , -2.1923268 , -1.5325681 , -2.135137  ,
       -1.1659501 ,  1.1294677 ,  0.35031214, -1.973543  ,  0.08085412,
        0.1273687 ,  1.345549  ,  0.9897706 ,  1.0814862 ,  0.16797668,
       -2.4001794 , -2.250866  , -2.1711917 ,  0.7540618 , -0.5087489 ,
        1.2697785 , -1.5433878 ,  0.52886194,  0.89852726,  2.8666606 ,
       -1.145751  , -1.4410472 , -1.0245936 , -0.22137724, -1.7295679 ,
       -0.7484694 , -0.9394028 ,  2.814444  , -0.37733868, -0.52629095,
       -2.0724132 , -1.1152315 , -0.924242  , -0.16516834, -0.8087195 ,
       -0.31686544, -0.11697809,  1.891504  , -2.6718369 ,  2.032406  ,
       -1.8871613 , -0.55631626,  1.1865749 ,  2.5178492 , -2.19

Now get all of the word vectors from the object at once. You can find these inside of `wv.vectors`. Try it out in the cell below.  

In [20]:
wv.vectors.shape

(154100, 100)

As a final exercise, try to recreate the _'king' - 'man' + 'woman' = 'queen'_ example previously mentioned. You can do this by using the `.most_similar()` method and translating the word analogies into an addition/subtraction formulation (as shown above). Pass the original comparison, which you are calculating a difference between, to the negative parameter, and the analogous starter you want to apply the same transformation to, to the `positive` parameter.

Do this now in the cell below. 

In [21]:
wv.most_similar(positive=['king', 'woman'], negative=['man'])

[('goddess', 0.5172680020332336),
 ('princess', 0.5068932771682739),
 ('queen', 0.5048187375068665),
 ('coin', 0.488680899143219),
 ('crown', 0.48776090145111084),
 ('purveyor', 0.48152655363082886),
 ('Queen', 0.47275036573410034),
 ('jewel', 0.4627821445465088),
 ('Michelangelo', 0.4620759189128876),
 ('epicenter', 0.45187222957611084)]

As you can see from the output above, your model isn't perfect, but 'Queen' and 'Princess' are still in the top 5. As you can see from the other word in top 5, 'reminiscent' -- your model is far from perfect. This is likely because you didn't have enough training data. That said, given the small amount of training data provided, the model still performs remarkably well! 

In the next lab, you'll reinvestigate transfer learning, loading in the weights from an open-sourced model that has already been trained for a very long time on a massive amount of data. Specifically, you'll work with the GloVe model from the Stanford NLP Group. There's not really any benefit from training the model ourselves, unless your text uses different, specialized vocabulary that isn't likely to be well represented inside an open-source model.

## Summary

In this lab, you learned how to train and use a Word2Vec model to create vectorized word embeddings!